### Synthetic data example

Here we're building a simple model with two confounders (race and gender) and a binary treatment of high vs low rank. We're generating a weight for each rank, and the difference between the two is our expected causal effect, and we generate our final outcome based off of the confounders and treatment.

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

In [2]:
def get_smf_model_a_param(ols, df):
    """
    Fit a model with statsmodels
    Return the parameter corresponding to the treatment
    """
    return smf.ols(ols, data=df).fit().params['a']

In [3]:
def get_sklearn_model_a_param(ols, df):
    """
    Fit a model with sklearn
    Return the parameter corresponding to the treatment
    """
    target = ols.split("~")[0].strip()
    inputs = ols.split("~")[1].strip().split(" + ")

    model = LinearRegression()
    model.fit(df[inputs], df[target])

    return model.coef_[inputs.index("a")]

### synthetic data generation for CPD project

In [8]:
def observed(n=100, ols="y ~ a + c1 + c2"):
    np.random.seed(88)
    # confounders = race (c1) and gender (c2)
    c1 = np.random.binomial(n=1, p=0.423, size=n)

    # choose p based on distribution of male vs female officers in the data
    c2 = np.random.binomial(n=1, p=0.8763, size=n)

    # low rank vs high rank
    # p = 0.3 + 0.2c1 + 0.2c2 (default prob of promotion is 0.3 but goes up w/ race/gender 1)
    a = np.random.binomial(n=1, p=0.3 + 0.2*c1 + 0.2 * c2, size=n)

    # hard to do w/ 7 bc want to fit with different models to see how well you're fitting counterfactuals
    # vector of weights to rep prob of suspension at each rank
    weights = np.random.uniform(0, 0.5, 2)
    weights.sort()
    weights = weights[::-1]
    print("weights: ", weights)
    print("expected causal effect: ", weights[1]-weights[0])

    y = []

    # relationship b/t a and y should be small weight - big weight
    for i in range(n):
      rank = a[i]
      p = (c1[i] + c2[i]) / 4 + weights[rank]
      p = min(p, 1)
      y.append(np.random.binomial(n=1, p=p))
    y = np.array(y)

    df = pd.DataFrame(data=dict(c1=c1, c2=c2, a=a, y=y))
    a_param = get_smf_model_a_param(ols, df)
    naive_a = get_smf_model_a_param("y ~ a", df)

    print("naive estimate: ", naive_a)
    # print("observed causal effect with backdoor: ", a_param)

    return a_param

In [13]:
print("observed causal effect with backdoor: ", observed(n=1000))

weights:  [0.15010261 0.00617352]
expected causal effect:  -0.14392909686181932
naive estimate:  -0.1010109308697455
observed causal effect with backdoor:  -0.17863670285842156


In [10]:
print("observed causal effect with backdoor: ", observed(n=10000))

weights:  [0.28190298 0.11372635]
expected causal effect:  -0.16817663430800922
naive estimate:  -0.09584415584415641
observed causal effect with backdoor:  -0.1660791639489527


In [11]:
print("observed causal effect with backdoor: ", observed(n=100000))

weights:  [0.45014063 0.29954051]
expected causal effect:  -0.1506001262063042
naive estimate:  -0.07708963852378756
observed causal effect with backdoor:  -0.14968755812279524
